# Ehrenfest Urns

In this exercise we are going to look at another famour Markov Chain problem known as the Ehrenfest Urns.  This problem involves a rather nice example of a ergodic markov chain that has a stationary distribution.  The statement of the problem looks something like this:

A mathematical model for the process of diffusion of a gas is known as the Ehrenfest urns.  A set of 6 numbered balls is divided (not necessarily equally) between urn A and urn B.  Each minute, a dice is rolled in order to select a random integer between 1 and 6.  The ball with this number is located in one of the urns and is then immediately transferred to the other urn.  The 'state' of the system at any time, $t$, is denoted by $X_t=i$, where $i$ is the number of balls in urn A at that time.

In this exercise we are going to write a program to simulate this process.  Before we get onto that though you first need to take some time to understand the problem.  Hence, in your notes on this exercise:

- Explain why the number of balls in urn A can be modelled using a Markov chain.
- Explain how many states there are in this Markov chain.
- Draw the transition graph for this Markov chain.
- Identify which states are transient and whcih states are recurrent in this Markov chain.
- Write out the transition matrix for this Markov chain.
- Calculate the period for each of the states in the chain

Once you have completed these tasks then move on to the remainder of the exercises contained herein.  As always these tasks begin with some code for dynamic plotting that I have written for you so please press shift and enter on the cell below to begin.

In [ ]:
import math
%matplotlib notebook
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from matplotlib import rc
from IPython.display import HTML
import random
rc('animation', html='none')

class plotobj(object):
    def __init__(self,ny,xmin,xmax):
        self.fig = plt.figure()
        self.ny = ny
        self.xmin = xmin
        self.xmax = xmax
        self.ax = plt.axes(xlim=(xmin,xmax), ylim=(0, 1))
    
    def setup(self):
        self.fno = 0
        xdata,ydata = [],[]
        for i in range(self.xmin,self.xmax):
            xdata.append(i)
        for j in range(0,self.ny):
            ydata.append([])
            for i in range(self.xmin,self.xmax):
                ydata[j].append(0)
    
        self.rects = []
        collist = ['r','g','b','r','g','b','r']
        for j in range(0,self.ny):
            self.rects.append( self.ax.bar(xdata,ydata[j], 0.1, color=collist[j] ) )
            for k in range(self.xmin,self.xmax) :
                xdata[k] += 0.1
                
    def run(self,data):
        xdata,ydata=data
        self.fno +=1
        self.fig.suptitle('frame number ' + str(self.fno), fontsize=14, fontweight='bold' )
        maxy,j=0,0
        for rbunch in self.rects :
            i=0
            for rect in rbunch :
                if self.ny==1 :
                    rect.set_height(ydata[i])
                    if( ydata[i] > maxy ) :
                        maxy = ydata[i]
                else :
                    rect.set_height(ydata[j,i])
                    if( ydata[j,i]>maxy ) :
                        maxy = ydata[j,i]
                i+=1
            j += 1
        self.ax.set_ylim(0,maxy+0.1)
        return self.rects

def dynamicplot( ny, ngen, operation, xmin, xmax, myvar, tinverval=1):
    myplot = plotobj(  ny, xmin, xmax )
    return anim.FuncAnimation(fig=myplot.fig, func=myplot.run, frames=operation( ngen, myvar ), 
                                init_func=myplot.setup, interval=10, blit=False, repeat=False)

# Introduction

In this first section we are going to simulate the behavior of the Markov chain in order to examine whether or not a limiting stationary distribution emerges if we run the chain over a long period of time.  Consequently, in the cell below I have once again provided you with code that you can use to generate a large number of random variables and to generate a histogram from these variables.  Press shift and enter on this cell to load all this code now.  

In [ ]:
class histoclass(object) :
    def __init__( self, nstates, initstate ):
        self.initstate=initstate
        self.xdata = []
        self.ycount = []
        for i in range(0,nstates) :
            self.xdata.append( i )
            self.ycount.append(0)
            
    def buildHistogram( self, ngen, myvar ) :
        cnt=0
        pstate = self.initstate    
        while cnt<ngen :
            cnt += 1
            # Notice that the random number generator function now 
            # depends on the value generated for the previous random variable 
            # This is the Markov property.
            X = myvar( pstate )   
            pstate = X
            self.ycount[X] += 1
            tdata = []
            for dat in self.ycount :
                tdata.append( dat / cnt )
            yield self.xdata, tdata

In the cell below I have written some of the code that you will need in order to generate the histogram.  In particular I have written the calls to dynamic plot that are required in order to build the dynamic histogram.  What I have not done, however, is to write the random number generator.  This is your task.  Modify the function genmove in the code block below so that it generates random moves in accordance with the Markov process (the Ehrenfest Urns) that I described in the introduction to this exercise.  Notice that this function takes the state that I am presently in (instate) as an argument and that it should return the state that I will be in on the next step.  Remember as you write this code that we are assuming that there are 6 balls that can be moved between the two cups.

In [ ]:
# You need to modify this function as it should 
# return a random variable.  At the moment it just
# generates the number 1, which is not very random.
def genmove( instate ):
    return 1

# Build an instance of the histogram object called myhisto
# the arguments here are the number of states in the chain (7)
# and the initial state (3)
myhisto = histoclass( 7, 3 )
# Now build a dynamic histogram that shows how the number of visits
# to each states changes as a function of time.  The arguments here are
# The number of series to plot = 1
# The number of random variables to generate = 1000
# The function to use to generate the frames = myhisto.buildHistogram
# The lower (0) and upper (7) bounds to use for the x axis
# The function that generates our random variables = genmove (see above)
dynamicplot( 1, 1000, myhisto.buildHistogram, 0, 7, genmove )

Once you have finished writing your function run the cell above and look at the graph that you generate.  Describe what happens to the histogram as the simulation progresses.  Based on what you observer does the chain have a limiting stationary distribution?

# The Chapman Kolmogorov Relation 

In the first exercise we did on ergodic Markov chains we first simulated the chain as we did in the previous section and then looked at the powers of the transition matrix.  When we did this second part we saw that, when we took large powers of the transition matrix, the rows of the resulting $n$-step transition matrix were all approximately equal.  Furthermore, we stated that this was a further manifestation of the existence of the limiting stationary distribution.  In this next section we are thus going to examine the powers of the transition matrix for the Ehrenfest Urn problem.  In the cell below you should thus set a python variable called $A$ equal to the transition matrix for the Ehrenfest Urn problem assuming that there are 6 balls that can be transferred between the two cups.   

In [ ]:
A = 1

When you run the following code the 7 rows of the transition matrix are plotted as a function of time.  The left-most bars at 0, 1, 2, 3,etc are the first row of the matrix, the next set of bars at 0.1, 1.1, 2.1, 3.1,etc are the second row, the bars at 0.2, 1.2, 2.2, 3.2, etc are the third row and so on up to the seventh row of the matrix.  I shoudl point out that there are thus seven bars for each number (0,1,2,3,4,5,6 and 7).  If it looks like no bar is present then the value of the corresponding transition probability is equal to zero.  Describe what you observe for the transition probabilities for this matrix.  How does what you observe here compare with what you observed when we looked at the three state Markov chain in the previous exercise? 

In [ ]:
class matrixobj(object):
    def __init__(self,nbins,tmatrix):
        self.tmatrix = tmatrix 
        self.xdata = []
        for i in range(0,nbins) :
            self.xdata.append(i)
        
    def calcpowers(self,ngen,myvar):
        cnt = 0
        pmatrix = self.tmatrix
        while cnt<ngen :
            cnt += 1
            # This part calculates the powers of the matrix
            pmatrix = np.dot( pmatrix, self.tmatrix )
            yield self.xdata,pmatrix

mypowers = matrixobj( 7, A )
dynamicplot( 7, 100, mypowers.calcpowers, 0, 7, genmove, 1000 )

# Periodicity and the stationary distribution

In the previous exercises you saw that it looks like this Markov chain has a limiting stationary distribution.  However, when we looked more closely at high powers of the matrix we found that the each row of the vector does not seem to tend to a single vector.  Instead each row of the matrix seems to oscillate between the following pair of vectors:

$$
\begin{aligned}
v_1 = \left( 
\begin{matrix} 
0 & \frac{3}{32} & 0 & \frac{5}{16} & 0 & \frac{3}{32} & 0
\end{matrix}
\right) & \qquad \textrm{and} & 
v_2 = \left(
\begin{matrix}
\frac{1}{32} & 0 & \frac{15}{64} & 0 & \frac{15}{64} & 0 & \frac{1}{32}
\end{matrix}
\right)
\end{aligned}
$$

In this section we are going to try to work out why this is happening.  With this in mind use the cell below to calculate the eigenvalues and eigenvectors of the transition matrix.  In your notes explain how the eigenvalues differ from those in the previous problem and why the powers of the matrix no longer converge to a single vector.  In answering this question you should make reference to the content of this video (https://www.youtube.com/watch?v=1v-GEdV8zys) again. 

# Conclusion

To be totally clear we can find a stationary distribution for this Markov chain by using the detailed balance condition:

$$
\pi_i p_{ij} = \pi_j p_{ji}
$$

As a final exercise in your notes use the above condition to find the stationary distribution for an Ehrenfest Urn Markov chain in which there are $N$ balls that can be transferred between the two cups.